In [1]:
#importing useful libraries
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import math
from types import SimpleNamespace

In [2]:
PARAMETERS = {

    #Knowns for Ground Truth
    'grndAlpha': 10,
    'grndMeanX': 0,
    'grndMeanY': 0,
    'grndSigmaX': 10,
    'grndSigmaY': 10,
    'grndMag': 100,
    'grndZ': None,

    #General Knowns
    'nDataX': 100,
    'nDataY': 100,
    'nInduX': 50,
    'nInduY': 50,
    'nFineX': 500,
    'nFineY': 500,
    'covLambda': 10,
    'covL': 1,

    #Knowns to be evaluated
    'data': None,
    'grndX': None,
    'grndY': None,
    'xIndu': None,
    'yIndu': None,
    'xFine': None,
    'yFine': None,
    'cInduIndu': None,
    'cInduData': None,
    'cInduFine': None,
    'cInduInduInv': None,

    # Variables
    'P': float,
    'alpha': float,
    'fIndu': None,

    # Priors
    'ell': 1,
    'sig': 10,
    'fInduMean': 0,
    'alphaShape': 1,
    'alphaScale': 1000,


    # Sampler parameters
    'epsilon': 1e-3,
}

# function to initialize all parameters that are not known or variable
def initialization(parameters):

    #declare variables as object of 
    variables = SimpleNamespace(**parameters)

    # extract knowns
    grndAlpha = variables.grndAlpha
    grndMeanX = variables.grndMeanX
    grndMeanY = variables.grndMeanY
    grndSigmaX = variables.grndSigmaX
    grndSigmaY = variables.grndSigmaY
    grndMag = variables.grndMag
    nDataX = variables.nDataX
    nDataY = variables.nDataY
    covLambda = variables.covLambda
    covL = variables.covL
    epsilon = variables.epsilon
    

    # sample Data
    variables.grndX, variables.grndY, variables.grndZ, variables.data = curveFinder.dataGenerator(
        nDataX, nDataY, grndMeanX, grndMeanY, grndSigmaX, grndSigmaY, grndMag, grndAlpha)
    
    #define x and y for Inducing points
    x = np.linspace(np.min(variables.grndX), np.max(variables.grndX), variables.nInduX)
    y = np.linspace(np.min(variables.grndY), np.max(variables.grndY), variables.nInduY)
    variables.xIndu, variables.yIndu = np.meshgrid(x, y)
    
    #define x and y for Fine points
    x = np.linspace(np.min(variables.grndX), np.max(variables.grndX), variables.nFineX)
    y = np.linspace(np.min(variables.grndY), np.max(variables.grndY), variables.nFineY)
    variables.xFine, variables.yFine = np.meshgrid(x, y)

    #detrmine Covarince matrices
    variables.cInduIndu = curveFinder.covMat(
        variables.xIndu, variables.yIndu, variables.xIndu, variables.yIndu, covLambda, covL)
    variables.cInduData = curveFinder.covMat(
        variables.xIndu, variables.yIndu, variables.grndX, variables.grndY, covLambda, covL)
    variables.cInduFine = curveFinder.covMat(
        variables.xIndu, variables.yIndu, variables.xFine, variables.yFine, covLambda, covL)
    variables.cInduInduInv = np.linalg.inv(variables.cInduIndu + epsilon*np.eye(variables.nInduX*variables.nInduY))
    
    
    #Initial sample for inducing points
    variables.fIndu = np.ones(variables.nInduX*variables.nInduY) * np.mean(variables.data)
    
    #Initial guess for alpha
    variables.alpha = 10
    
    #Is this next line necassary?
    variables.alphaScale = np.mean(variables.data)/variables.alphaShape

    return variables
 
def curveSampler(variables):

    #necassary variables
    nIndu = variables.nInduX*variables.nInduY
    cInduIndu = variables.cInduIndu
    cInduData = variables.cInduData
    cInduInduInv = variables.cInduInduInv
    alpha = variables.alpha
    data = variables.data.reshape(variables.nDataX*variables.nDataY, 1)
    
    for i in range(np.random.randint(1,100)):
        
        # Propose new fIndu
        fInduOld = variables.fIndu
        fInduNew = fInduOld + np.random.multivariate_normal(mean = np.zeros(nIndu), cov = cInduIndu)

        # Calculate probabilities of induced samples
        def probability(fIndu):

            # Prior
            prior = stats.multivariate_normal.logpdf(fIndu, mean = np.zeros(nIndu), cov=cInduIndu)

            #grnd of data associated with fIndu
            fData = cInduData.T @ cInduInduInv @ fIndu
            
            #Likelihood of that data
            lhood = np.sum(stats.gamma.logpdf(data, a = alpha, scale = fData/alpha))

            prob = lhood + prior

            return prob

        #Probability of old and new function
        pOld = probability(fInduOld)
        pNew = probability(fInduNew)
        
        #Acceptance value
        acc_prob = pNew - pOld

        if np.log(np.random.rand()) < acc_prob:
            variables.fIndu = fInduNew

    return variables
'''
def fineCurveSampler(variables):

    #necassary variables
    nIndu = variables.nIndu
    cInduIndu = variables.cInduIndu
    cInduData = variables.cInduData
    cInduInduInv = variables.cInduInduInv
    alpha = variables.alpha
    data = variables.data.reshape(variables.nDataX*variables.nDataY, 1)
    
    for i in range(np.random.randint(1,100)):
        # Propose new fIndu
        fInduOld = variables.fIndu
        fInduNew = fInduOld + np.random.multivariate_normal(mean = np.zeros(nIndu), cov = cInduIndu)*.1

        # Calculate probabilities of induced samples
        def probability(fIndu):

            # Prior
            prior = stats.multivariate_normal.logpdf(fIndu, mean = np.zeros(nIndu), cov=cInduIndu)

            #grnd of data associated with fIndu
            fData = cInduData.T @ cInduInduInv @ fIndu
            
            #Likelihood of that data
            lhood = np.sum(stats.gamma.logpdf(data, a = alpha, scale = fData/alpha))

            prob = lhood + prior

            return prob

        #Probability of old and new function
        pOld = probability(fInduOld)
        pNew = probability(fInduNew)
        
        #Acceptance value
        acc_prob = pNew - pOld

        if np.log(np.random.rand()) < acc_prob:
            variables.fIndu = fInduNew

    return variables

def finerCurveSampler(variables):

    #necassary variables
    nIndu = variables.nIndu
    cInduIndu = variables.cInduIndu
    cInduData = variables.cInduData
    cInduInduInv = variables.cInduInduInv
    alpha = variables.alpha
    data = variables.data
    
    for i in range(np.random.randint(1,1000)):
        # Propose new fIndu
        fInduOld = variables.fIndu
        fInduNew = fInduOld + np.random.multivariate_normal(mean = np.zeros(nIndu), cov = cInduIndu)*.001

        # Calculate probabilities of induced samples
        def probability(fIndu):

            # Prior
            prior = stats.multivariate_normal.logpdf(fIndu, mean = np.zeros(nIndu), cov=cInduIndu)

            #grnd of data associated with fIndu
            fData = cInduData.T @ cInduInduInv @ fIndu
            
            #Likelihood of that data
            lhood = np.sum(stats.gamma.logpdf(data, a = alpha, scale = fData/alpha))

            prob = lhood + prior

            return prob

        #Probability of old and new function
        pOld = probability(fInduOld)
        pNew = probability(fInduNew)
        
        #Acceptance value
        acc_prob = pNew - pOld

        if np.log(np.random.rand()) < acc_prob:
            variables.fIndu = fInduNew

    return variables
'''
def alphaSampler(variables):

    # necassary values
    cInduData = variables.cInduData
    cInduInduInv = variables.cInduInduInv
    fIndu = variables.fIndu
    data = variables.data.reshape(variables.nDataX*variables.nDataY, 1)

    #grnd of data associated with fIndu
    fData = cInduData.T @ cInduInduInv @ fIndu

    for i in range(np.random.randint(1,1000)):
        # Propose new alpha
        alphaOld = variables.alpha
        alphaNew = alphaOld + np.random.randn() * 0.1

        while (alphaNew < 0 or alphaNew > 100):
            alphaNew = alphaOld + np.random.randn() * 0.1

        # Calculate probabilities of induced samples
        def probability(alpha):

            # Prior
            prior = np.log(0.01)

            #Likelihood of that data
            lhood = np.sum(stats.gamma.logpdf(data, a=alpha, scale=fData/alpha))

            #probability
            prob = lhood + prior

            return prob

        # Acceptance value
        pOld = probability(alphaOld)
        pNew = probability(alphaNew)

        #Acceptance value
        acc_prob = pNew - pOld

        # Update alpha
        if np.log(np.random.rand()) < acc_prob:
            variables.alpha = alphaNew

    return variables

def liklihood(variables):

    # Necassary values
    cInduData = variables.cInduData
    cInduInduInv = variables.cInduInduInv
    fIndu = variables.fIndu
    alpha = variables.alpha
    data = variables.data.reshape(variables.nDataX*variables.nDataY, 1)

    # grnd of data associated with fIndu
    fData = cInduData.T @ cInduInduInv @ fIndu

    # Likelihood
    lhood = np.sum(stats.gamma.logpdf(data, a = alpha, scale = fData/alpha))

    return lhood

class curveFinder:
    
    '''This method creates synthetic data assuming gamma noise on a function with gaussian form'''
    @staticmethod
    def dataGenerator(lengthx, lengthy, meanx, meany, sigmax, sigmay, mag, alpha):
        
        def function(X, Y, meanx, meany, sigmax, sigmay):
            xterm = np.exp(-((X - meanx)**2)/(2*sigmax**2))
            yterm = np.exp(-((Y - meany)**2)/(2*sigmay**2))
            z = mag*xterm*yterm
            return z

        x = np.linspace(-10, 10, lengthx)
        y = np.linspace(-10, 10, lengthy)

        X, Y = np.meshgrid(x, y)
        
        zground = function(X, Y, meanx, meany, sigmax, sigmay)

        data = np.random.gamma(alpha, zground/alpha, np.shape(zground))
        return X, Y, zground, data

    '''This is a function that returns a covariance matrix between two position vectors
        based on square exponential kernal with parameters covLambda and covL'''    
    @staticmethod
    def covMat(x1, y1, x2, y2, covLambda, covL):
        
        #reshape x and y to be column vectors
        x1 = x1.reshape(np.shape(x1)[0]*np.shape(x1)[1], 1)
        y1 = y1.reshape(np.shape(y1)[0]*np.shape(y1)[1], 1)
        x2 = x2.reshape(np.shape(x2)[0]*np.shape(x2)[1], 1)
        y2 = y2.reshape(np.shape(y2)[0]*np.shape(y2)[1], 1)

        #Create empty matrix for covariance
        C = np.zeros((len(x1),len(x2)))

        #loop over all indecies in covariance matrix
        for i in range(len(x1)):
            for j in range(len(x2)):
                #Calculate distance between points
                dist = math.sqrt((x1[i] - x2[j])**2 + (y1[i] - y2[j])**2)
                #Determine each element of covariance matrix
                C[i][j] = (covLambda**2)*(math.exp(((-1)*((dist)**2))/(covL**2)))

        #Return Covariance Matrix
        return C
   

In [3]:
#parameters = {**PARAMETERS, **parameters}

# Initialize variables
variables = initialization(PARAMETERS)
plt.contour(variables.grndX, variables.grndY, variables.grndZ, colors = 'k')
plt.contour(variables.grndX, variables.grndY, variables.data, colors = 'r')

KeyboardInterrupt: 

In [ ]:
fData = variables.cInduData.T @ variables.cInduInduInv @ variables.fIndu

np.shape(variables.data)

In [ ]:
def GibbsSampler(variables):

    alphaVect = []
    alphaVect.append(variables.alpha)

    curveVect = []
    curveVect.append(variables.fIndu)

    pVect = []

    for i in range(10):
        
        #variables = alphaSampler(variables)
        alphaVect.append(variables.alpha)

        variables = curveSampler(variables)
        curveVect.append(variables.fIndu)

        #variables.P = liklihood(variables)
        #pVect.append(variables.P)
    '''
    for i in range(25):
        
        variables = alphaSampler(variables)
        alphaVect.append(variables.alpha)

        variables = fineCurveSampler(variables)
        curveVect.append(variables.fIndu)

        variables.P = liklihood(variables)
        pVect.append(variables.P)

    for i in range(100):
        
        variables = alphaSampler(variables)
        alphaVect.append(variables.alpha)

        variables = finerCurveSampler(variables)
        curveVect.append(variables.fIndu)

        variables.P = liklihood(variables)
        pVect.append(variables.P)
    '''
    return variables, alphaVect, curveVect, pVect

variables, alphaVect, curveVect, pVect = GibbsSampler(variables)


In [ ]:
np.shape(variables.yIndu)[0]*np.shape(variables.yIndu)[1]

In [ ]:
fData = variables.cInduData.T @ variables.cInduInduInv @ variables.fIndu
plt.contour(variables.grndX, variables.grndY, variables.grndZ)
plt.contour(variables.grndX, variables.grndY, fData.reshape(np.shape(variables.grndX)))

fig = plt.figure()
ax = plt.axes(projection='3d')
x = variables.xIndu.reshape(np.shape(variables.xIndu)[0]*np.shape(variables.xIndu)[1],1)
y = variables.yIndu.reshape(np.shape(variables.yIndu)[0]*np.shape(variables.yIndu)[1],1)
z = variables.fIndu.reshape(np.shape(variables.yIndu)[0]*np.shape(variables.yIndu)[1],1)
ax.plot3D(x, y, z, 'gray')